In [4]:
from queue import PriorityQueue
import heapq
import random
import time
import threading
import math

In [2]:
class Maze:
    def __init__(self, num):
        self.size = num
        self.maze = [[] for _ in range(self.size)]

    def Q1(self, source, targets):
        dist = {i: float('inf') for i in range(self.size)}
        parent = {i: None for i in range(self.size)} 
        dist[source] = 0
        q = PriorityQueue()
        q.put((0, source)) 
        
        while not q.empty():
            d, u = q.get()
            
            if all(dist[t] < float('inf') for t in targets):
                break
            
            for v, cost in self.maze[u]:
                if dist[u] + cost < dist[v]:
                    dist[v] = dist[u] + cost
                    parent[v] = u  
                    q.put((dist[v], v))
        
        def getAns(target):
            path = []
            while target is not None:
                path.append(target)
                target = parent[target]
            return path[::-1]  
        
        print("Paths:")
        for target in targets:
            if dist[target] == float('inf'):
                print(f"No path to {target}")
            else:
                print(f"Path to {target}: {' -> '.join(map(str, getAns(target)))}")

    def addedge(self, x, y, cost):
        self.maze[x].append((y, cost))

G = Maze(16)
G.addedge(0, 1, 3)
G.addedge(0, 2, 6)
G.addedge(0, 3, 5)
G.addedge(1, 4, 2)  # Edge Added to showcase that it backtracks from deadend
G.addedge(2, 6, 12)
G.addedge(2, 7, 14)
G.addedge(3, 8, 7)
G.addedge(8, 9, 5)
G.addedge(8, 10, 6)
G.addedge(9, 11, 1)
G.addedge(9, 12, 10)
G.addedge(9, 13, 2)

source = 0
targets = [9, 4, 12] 
G.Q1(source, targets)

Paths:
Path to 9: 0 -> 3 -> 8 -> 9
Path to 4: 0 -> 1 -> 4
Path to 12: 0 -> 3 -> 8 -> 9 -> 12


In [ ]:
class Graph:    
    def __init__(self):
        self.nodes = set()
        self.edges = {}
        
    def addNode(self, value):
        self.nodes.add(value)
        if value not in self.edges:
            self.edges[value] = {}
            
    def addEdge(self, Source, Dest, cost):
        self.edges[Source][Dest] = cost
        self.edges[Dest][Source] = cost

    def neighbors(self,node):
        return self.edges.get(node, {}).keys()

    def getCost(self, Source, Dest):
        return self.edges[Source].get(Dest, float('inf'))
    

    def update(self, Source, Dest, new):
        if Source in self.edges and Dest in self.edges[Source]:
            self.edges[Source][Dest] = new
            self.edges[Dest][Source] = new

def heu(node, goal):
    x1, y1 = node
    x2, y2 = goal
    return abs(x1 - x2) + abs(y1 - y2)

def AStar(graph, start, goal):
    oSet = []
    heapq.heappush(oSet, (0, start))
    parent = {}
    gScore = {node: float('inf') for node in graph.nodes}
    gScore[start] = 0
    fScore = {node: float('inf') for node in graph.nodes}
    fScore[start] = heu(start, goal)
    
    while oSet:
        _, curr = heapq.heappop(oSet)
        
        if curr == goal:
            path = []
            while curr in parent:
                path.append(curr)
                current = parent[curr]
            path.append(start)
            return path[::-1]
        

        for neighbor in graph.neighbors(curr):
            tScore = gScore[curr] + graph.getCost(curr, neighbor)
            if tScore < gScore[neighbor]:
                parent[neighbor] = curr
                gScore[neighbor] = tScore
                fScore[neighbor] = tScore + heu(neighbor, goal)
                heapq.heappush(oSet,(fScore[neighbor], neighbor))
    
    return None  

graph = Graph()
nodes = [(x, y) for x in range(5) for y in range(5)]
for node in nodes:
    graph.addNode(node)

for x in range(5):
    for y in range(5):
        if x < 4:
            graph.addEdge((x, y), (x+1, y), random.randint(1, 5))
        if y < 4:
            graph.addEdge((x, y), (x, y+1), random.randint(1, 5))

start, goal = (0, 0), (4, 4)

for i in range(20): 
    if i % 3 == 0: 
        node1, node2 = random.sample(nodes, 2)
        new = random.randint(1, 10)
        graph.update(node1, node2, new)
        print(f"Edge cost updated between {node1} - {node2} to {new}")

    path = AStar(graph, start, goal)
    print(f"Iteration {i+1}: Optimal Path: {path}")

    time.sleep(2)  

Edge cost updated between (2, 0) - (1, 2) to 1


In [9]:
class Deliverys:
    def __init__(self, id, x , y, t):
        self.id = id
        self.xAxis = x
        self.yAxis = y
        self.time = t
    
    def distance(self, sec):
        return math.sqrt((self.x - sec.x) ** 2 + (self.y - sec.y) ** 2)


def GBFS(start, Points):
    route = []
    curr=start
    q = []
    
    for point in Points:
        heapq.heappush(q, (point.time[1], point))
    

    while q:
        _, next = heapq.heappop(q)
        route.append(next)
        curr = next
    return route


Pts = [
    Deliverys(1001, 3, 4, (9, 11)),
    
    Deliverys(1020, 7, 2, (6, 8)),

    Deliverys(1003, 5, 6, (10, 12)),

    Deliverys(1045, 2, 8, (7, 9))
]

start = Deliverys(1000, 0, 0, (0, 24))
opRoute = GBFS(start, Pts)

print("Route:")
for point in opRoute:
    print(f"Delivering at Point {point.id} located at ({point.xAxis}, {point.yAxis}) within time window {point.time}")

Route:
Delivering at Point 1020 located at (7, 2) within time window (6, 8)
Delivering at Point 1045 located at (2, 8) within time window (7, 9)
Delivering at Point 1001 located at (3, 4) within time window (9, 11)
Delivering at Point 1003 located at (5, 6) within time window (10, 12)


In [16]:
def heu(first, second):
    return abs(first[0] - second[0]) + abs(first[1] - second[1])


def Q4(graph, start, goal, tStatus):
    openn = []
    heapq.heappush(openn, (0, start))

    parent = {}

    gScore = {node: float('inf') for node in graph}
    gScore[start] = 0
    fScore={node: float('inf') for node in graph}
    fScore[start]=heu(start, goal)
    


    while openn:
        _, curr = heapq.heappop(openn)
        if curr == goal:
            path = []
            while curr in parent:
                path.append(curr)
                curr = parent[curr]
            path.append(start)
            return path[::-1]
        
        for neighbor in graph[curr]:
            updweight = round(graph[curr][neighbor] * tStatus.get((curr, neighbor), 1),3)

            Tgscore = gScore[curr] + updweight
            if Tgscore < gScore[neighbor]:

                parent[neighbor] = curr
                gScore[neighbor]=round(Tgscore,3)
                fScore[neighbor]=gScore[neighbor] + round(heu(neighbor, goal),3)
                heapq.heappush(openn, (round(fScore[neighbor], 3), neighbor))
    return None


def update(graph):
    updates = {}
    for node in graph:
        for neighbor in graph[node]:
            updates[(node, neighbor)] = round(random.uniform(0.5, 2),3)
    return updates



roads = {
    (0, 0): {(0, 1): 2, (1, 0): 5},
    (0, 1): {(0, 0): 2, (1, 1): 3, (0, 2): 4},
    (0, 2): {(0, 1): 4, (1, 2): 2},
    (1, 0): {(0, 0): 5, (1, 1): 2},
    (1, 1): {(1, 0): 2, (0, 1): 3, (1, 2): 3},
    (1, 2): {(0, 2): 2, (1, 1): 3},
}

s = (0, 0)
goal = (1, 2)
traf = update(roads)

shortestPath = Q4(roads, s , goal, traf)

print("after update:", traf)
print("optimal:", shortestPath)

after update: {((0, 0), (0, 1)): 1.314, ((0, 0), (1, 0)): 1.201, ((0, 1), (0, 0)): 1.401, ((0, 1), (1, 1)): 1.08, ((0, 1), (0, 2)): 1.475, ((0, 2), (0, 1)): 0.752, ((0, 2), (1, 2)): 0.652, ((1, 0), (0, 0)): 1.035, ((1, 0), (1, 1)): 0.91, ((1, 1), (1, 0)): 0.585, ((1, 1), (0, 1)): 1.735, ((1, 1), (1, 2)): 1.561, ((1, 2), (0, 2)): 0.566, ((1, 2), (1, 1)): 1.24}
optimal: [(0, 0), (0, 1), (0, 2), (1, 2)]
